# Libs import

In [7]:
# load requires packages
pacman::p_load(MendelianRandomization, TwoSampleMR, MVMR,
               ieugwasr, gwasglue,
               data.table, plyr, dplyr, tidyr, meta, SciViews, qqman, comprehenr, cli, knitr,
               ggplot2, cowplot, plotly, ggrepel, egg, grid, gridExtra,
               optparse, metaCCA)
options(ieugwasr_api = 'gwas-api.mrcieu.ac.uk/')

# Global Variables

In [10]:
global_seed <- 314159265
output_dir <- "output/" # path where the results are saved
data_dir <- "data" # path to where you have the datasets saved

In [11]:
snpList_t2d <- read.csv(paste0(data_dir,"/paper-results/snpList_t2d.csv"))[,1]
snplist_bf <- read.csv(paste0(data_dir,"/paper-results/snpList_bf.csv"), header = FALSE)[-1,1]
snplist_bmi <- read.csv(paste0(data_dir,"/paper-results/snpList_bmi.csv"), header = FALSE)[-1,1]

ld_ref_file <- paste0(data_dir, "/LD_ref/EUR")
ld_mat_uk10k_file <- paste0(data_dir, "/LD_ref/UK10K_COHORT.20160215.sites.tab.gz")
plink_path <- genetics.binaRies::get_plink_binary()

full_method_list <- c("mr_egger_regression", "mr_weighted_median", "mr_ivw", "mr_ivw_fe", "mr_ivw_radial", "mr_raps")

In [12]:
id_als_2018 <- "ebi-a-GCST005647"
id_als_2021 <- "ebi-a-GCST90027164"
id_park_2019 <- "ieu-b-7"
id_chd <- "ieu-a-7"

id_t2d_2018 <- "ebi-a-GCST006867"
id_t2d_2021 <- "ebi-a-GCST90018926"
id_tc_2013 <- "ebi-a-GCST002221"
id_tc_2021 <- "ebi-a-GCST90018974"
id_bmi_2018a <- "ebi-a-GCST90029007"
id_bmi_2018b <- "ieu-b-40"

id_hdlc <- "ieu-a-299"
id_ldlc <- "ieu-a-300"
id_trig <- "ieu-a-302"

In [13]:
# from https://www.ebi.ac.uk/gwas/studies/GCST005647
als_file_2018 <- paste0(data_dir, "/gwas/ALS/29566793-GCST005647-EFO_0000253.h.tsv.gz")
# from https://www.ebi.ac.uk/gwas/studies/GCST90027164 (meilleur)
als_file_2021 <- paste0(data_dir, "/gwas/ALS/GCST90027164_buildGRCh37.tsv.gz")
# from https://www.ebi.ac.uk/gwas/studies/GCST009325
park_file_2019 <- paste0(data_dir, "/gwas/Park/GCST009325.h.tsv.gz")
# from https://www.ebi.ac.uk/gwas/studies/GCST90275127 (meilleur)
park_file_2023 <- paste0(data_dir, "/gwas/Park/GCST90275127.h.tsv.gz")

# from https://www.ebi.ac.uk/gwas/studies/GCST003116
chd_file <- paste0(data_dir, "/gwas/CHD/26343387-GCST003116-EFO_0000378.h.tsv.gz")

# from https://www.ebi.ac.uk/gwas/studies/GCST006867 (meilleur)
t2d_file_2018a <- paste0(data_dir, "/gwas/T2D/Xue_et_al_T2D_META_Nat_Commun_2018.gz")
# from http://diagram-consortium.org/downloads.html
t2d_file_2018b <- paste0(data_dir, "/gwas/T2D/Mahajan.NatGenet2018b.T2D.European.txt.gz")
# from https://www-ncbi-nlm-nih-gov.insb.bib.cnrs.fr/projects/gap/cgi-bin/analysis.cgi?study_id=phs001672.v11.p1&pha=4945
t2d_file_2020 <- paste0(data_dir, "/gwas/T2D/phs001672.pha004945.csv.gz")
# from https://t2d.hugeamp.org/dinspector.html?dataset=Suzuki2024_T2D_EU
t2d_file_2024 <- paste0(data_dir, "/gwas/T2D/EUR_Metal_LDSC-CORR_Neff.v2.txt.gz")

#from https://www.ebi.ac.uk/gwas/publications/24097068 (meilleur)
tc_file_2013 <- paste0(data_dir, "/gwas/TC/24097068-GCST002221-EFO_0004574.h.tsv.gz")
ldlc_file_2013 <- paste0(data_dir, "/gwas/LDLC/24097068-GCST002222-EFO_0004611.h.tsv.gz")
# from https://www.ebi.ac.uk/gwas/studies/GCST90239676 
tc_file_2021 <- paste0(data_dir, "/gwas/TC/GCST90239676.h.tsv.gz")
# from https://www.ebi.ac.uk/gwas/studies/GCST90239658
ldlc_file_2021 <- paste0(data_dir, "/gwas/LDLC/GCST90239658_buildGRCh37.tsv.gz")

# from https://www.ebi.ac.uk/gwas/studies/GCST002783
bmi_file_2015 <- paste0(data_dir, "/gwas/BMI/25673413-GCST002783-EFO_0004340.h.tsv.gz")
# from https://zenodo.org/records/1251813
bmi_file_2018a <- paste0(data_dir, "/gwas/BMI/bmi.giant-ukbb.meta-analysis.combined.23May2018.txt.gz")
# from https://www.ebi.ac.uk/gwas/studies/GCST006900 (meilleur)
bmi_file_2018b <- paste0(data_dir, "/gwas/BMI/Meta-analysis_Locke_et_al+UKBiobank_2018_UPDATED.txt.gz")

# from https://www.ebi.ac.uk/gwas/studies/GCST003435
bf_file_2016 <- paste0(data_dir, "/gwas/BF%/26833246-GCST003435-EFO_0007800.h.tsv.gz")

# from https://www.ebi.ac.uk/gwas/studies/GCST90025952
alpb_file_2021 <- paste0(data_dir, "/gwas/ALPB/34226706-GCST90025952-EFO_0004615.h.tsv.gz")

# from https://www.ebi.ac.uk/gwas/studies/GCST90041837
hc_file <- paste0(data_dir, "/gwas/HC/29892013-GCST90029030-EFO_0007822.h.tsv.gz")

# from https://www.thessgac.org/#!data/kuzq8
ea_file <- paste0(data_dir, "/gwas/EA/GWAS_EA_excl23andMe.txt.gz")

# from TwoSampleMR api
hdlc_file_api <- paste0(data_dir, "/gwas/HDLC/24097068-GCST002223-EFO_0004612.h.tsv.gz")
ldlc_file_api <- ldlc_file_2013
trig_file_api <- paste0(data_dir, "/gwas/TRIG/24097068-GCST002216-EFO_0004530.h.tsv.gz")

In [14]:
##
std_useful_cols <- c("chromosome", "base_pair_location", "effect_allele", "other_allele", "effect_allele_frequency", "beta", "standard_error", "p_value", "id", "Phenotype")
hm_useful_cols <- c("hm_rsid", "hm_chrom", "hm_pos", "hm_effect_allele", "hm_other_allele", "hm_effect_allele_frequency", "hm_beta", "standard_error", "p_value", "id", "Phenotype")
##

als_useful_cols_2018 <- c("hm_rsid", std_useful_cols)
als_useful_cols_2021 <- c("rsid", std_useful_cols)

park_useful_cols_2019 <- c("rsid", std_useful_cols)
park_useful_cols_2023 <- c("rsid", std_useful_cols)

chd_useful_cols <- hm_useful_cols

t2d_useful_cols_2018a <- c("SNP", "CHR", "BP", "A1", "A2", "frq_A1", "b", "se", "P", "id", "Phenotype", "N")
t2d_useful_cols_2018b <- c("SNP", "Chr", "Pos", "EA", "NEA", "EAF", "Beta", "SE", "Pvalue", "id", "Phenotype")
t2d_useful_cols_2020 <- c("SNP.ID", "Chr.ID", "Chr.Position", "Allele1", "Allele2", "effect_allele_frequency", "beta", "SE", "P.value", "id", "Phenotype", "Sample.size")

tc_useful_cols_2013 <- hm_useful_cols
ldlc_useful_cols_2013 <- hm_useful_cols
tc_useful_cols_2021 <- c("variant_id",  std_useful_cols, "n")
ldlc_useful_cols_2021 <- c("variant_id", std_useful_cols)

bmi_useful_cols_sumstat <- hm_useful_cols
#bmi_useful_cols_2018a <- 
bmi_useful_cols_2018b <- c("SNP", "CHR", "POS", "Tested_Allele", "Other_Allele", "Freq_Tested_Allele_in_HRS", "BETA", "SE", "P", "id", "Phenotype", "N")

bf_useful_cols_sumstat <- hm_useful_cols

alpb_useful_cols_2021 <- hm_useful_cols

hc_useful_cols <- hm_useful_cols

ea_useful_cols <- c("MarkerName", "CHR", "POS", "A1", "A2", "EAF", "Beta", "SE", "Pval", "id", "Phenotype")

# Pre-Processing data (add id and trait name)

In [20]:
temp_file <- t2d_file_2020
temp_id <- "32541925-GCST010555"
temp_pheno <- "Type 2 diabetes"

In [21]:
temp_df <- fread(temp_file)

In [23]:
glimpse(temp_df)

Rows: 655,557
Columns: 14
$ `SNP ID`                <chr> "rs553642122", "rs7531543", "rs59192718", "rs7…
$ `P-value`               <dbl> 0.009862, 0.009649, 0.002186, 0.001852, 0.0061…
$ `Chr ID`                <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1…
$ `Chr Position`          <int> 790020, 917062, 922272, 920568, 917749, 920661…
$ Allele1                 <chr> "T", "C", "G", "A", "A", "T", "C", "G", "T", "…
$ Allele2                 <chr> "C", "A", "C", "G", "G", "C", "T", "C", "C", "…
$ `Minor allele`          <chr> "T", "C", "G", "A", "A", "T", "C", "G", "T", "…
$ beta                    <dbl> 0.1600, 0.1489, 0.2168, 0.2175, 0.1578, 0.1932…
$ SE                      <dbl> 0.0620, 0.0575, 0.0708, 0.0699, 0.0576, 0.0687…
$ `Coded Allele`          <chr> "T", "C", "G", "A", "A", "T", "C", "G", "T", "…
$ `Sample size`           <int> 826006, 1023280, 1023280, 1023280, 1023280, 10…
$ id                      <chr> "32541925-GCST010555", "32541925-GCST010555", …
$ Phenotype   

In [24]:
print_col_names(temp_df)

"SNP.ID", "P.value", "Chr.ID", "Chr.Position", "Allele1", "Allele2", "Minor.allele", "beta", "SE", "Coded.Allele", "Sample.size", "id", "Phenotype", "effect_allele_frequency"

In [19]:
add_pheno_id_in_file(temp_file, temp_df, temp_id, temp_pheno)

# Load scripts

In [102]:
source("scripts/00_utilitary.R")
source("scripts/01_data_import.R")
source("scripts/02_MR_methods.R")
source("scripts/03_plotting.R")
source("scripts/04_run_MR.R")
# from https://github.com/MRCIEU/PhenoSpD
# source("scripts/SpD.r")

# MR ALS

## Type 2 diabetes

In [ ]:
res_list_t2d <- global_uv(api_use_ex = FALSE, api_use_out = FALSE, 
                            exposure_id = t2d_file_2018a, outcome_id = als_file_2018, 
                            exposure_useful_cols = t2d_useful_cols_2018a, 
                            outcome_useful_cols = als_useful_cols_2018,
                            snplist = snpList_t2d, paper_results = "t2d_results.csv")

## Total Cholesterol

In [ ]:
res_tc_2021 <- global_uv(api_use_ex = FALSE, api_use_out = FALSE, 
                            exposure_id = tc_file_2013, outcome_id = als_file_2021, 
                            exposure_useful_cols = tc_useful_cols_2013, 
                            outcome_useful_cols = als_useful_cols_2021,
                            paper_results = "tc_results.csv", method_list = full_method_list)

## BodyFat%

In [ ]:
res_bf <- global_uv(api_use_ex = FALSE, api_use_out = FALSE,
                    exposure_id = bf_file_2016, outcome_id = als_file_2018, 
                    exposure_useful_cols = bf_useful_cols_sumstat, outcome_useful_cols = als_useful_cols_2018, 
                    snplist = snplist_bf, paper_results = "bf_results.csv")

# MR Parkinson

## BMI

In [ ]:
res_bmi_park <- global_uv(api_use_ex = FALSE, api_use_out = FALSE, 
                          exposure_id = bmi_file_2018b, outcome_id = park_file_2019, 
                          exposure_useful_cols = bmi_useful_cols_2018b, 
                          outcome_useful_cols = park_useful_cols_2019)

## T2D

In [ ]:
res_t2d_park <- global_uv(api_use_ex = FALSE, api_use_out = FALSE, 
                          exposure_id = t2d_file_2018a, outcome_id = park_file_2019, 
                          exposure_useful_cols = t2d_useful_cols_2018a, 
                          outcome_useful_cols = park_useful_cols_2019)

## TC

In [ ]:
res_tc_park <- global_uv(api_use_ex = FALSE, api_use_out = FALSE, 
                          exposure_id = tc_file_2021, outcome_id = park_file_2019, 
                          exposure_useful_cols = tc_useful_cols_2021, 
                          outcome_useful_cols = park_useful_cols_2019)

# MultiVariate MR

In [ ]:
# Diabete + Chol on ALS (Local)
res_t2d_tc <- global_mv(run_all_uvmr = TRUE, run_mvmr = TRUE, api_use = FALSE,
                            exposure_id_list = c(t2d_file_2018a, tc_file_2013), 
                            outcome_id = als_file_2021,
                            exposure_useful_cols_list = data.frame(t2d_useful_cols_2018a, tc_useful_cols_2013),
                            outcome_useful_cols = als_useful_cols_2021)

In [ ]:
res_t2d_tc_bmi <- global_mv(run_all_uvmr = TRUE, run_mvmr = TRUE, api_use = FALSE,
                            exposure_id_list = c(bmi_file_2018b, t2d_file_2018a, tc_file_2021),
                            exposure_useful_cols_list = data.frame(bmi_useful_cols_2018b, t2d_useful_cols_2018a, tc_useful_cols_2021),
                            outcome_id = als_file_2021, 
                            outcome_useful_cols = als_useful_cols_2021, run_mr_presso = TRUE)

In [ ]:
res_lipids_local <- global_mv(run_all_uvmr = FALSE, run_mvmr = TRUE, api_use = FALSE,
                                exposure_id_list = c(hdlc_file_api, ldlc_file_api, trig_file_api), 
                                outcome_id = chd_file,
                                exposure_useful_cols_list = data.frame(hm_useful_cols, hm_useful_cols, hm_useful_cols),
                                outcome_useful_cols = chd_useful_cols)

In [ ]:
res_lipids2_local <- global_mv(run_all_uvmr = TRUE, run_mvmr = TRUE, api_use = FALSE,
                                exposure_id_list = c(ldlc_file_api, trig_file_api, alpb_file_2021), 
                                outcome_id = chd_file,
                                exposure_useful_cols_list = data.frame(hm_useful_cols, hm_useful_cols, hm_useful_cols),
                                outcome_useful_cols = chd_useful_cols)

# Final results and plots

## Generate data

In [ ]:
# T2D, TC, BMI on ALS (All SNP without second clumping)
dat_als_all <- data_import_mv(api_use = FALSE, second_clumping = FALSE,
                                exposure_id_list = c(bmi_file_2018b, t2d_file_2018a, tc_file_2021),
                                exposure_useful_cols_list = data.frame(bmi_useful_cols_2018b, t2d_useful_cols_2018a, tc_useful_cols_2021),
                                outcome_id = als_file_2021, outcome_useful_cols = als_useful_cols_2021)
save(dat_als_all, file = "data_formatted/dat_als_all.RData")

In [ ]:
# T2D, TC, BMI on ALS 
dat_als <- data_import_mv(api_use = FALSE,
                            exposure_id_list = c(bmi_file_2018b, t2d_file_2018a, tc_file_2021),
                            exposure_useful_cols_list = data.frame(bmi_useful_cols_2018b, t2d_useful_cols_2018a, tc_useful_cols_2021),
                            outcome_id = als_file_2021, outcome_useful_cols = als_useful_cols_2021)
save(dat_als, file = "data_formatted/dat_als.RData")

In [ ]:
# T2D, TC, BMI on Park 
dat_park <- data_import_mv(api_use = FALSE,
                            exposure_id_list = c(bmi_file_2018b, t2d_file_2018a, tc_file_2021),
                            exposure_useful_cols_list = data.frame(bmi_useful_cols_2018b, t2d_useful_cols_2018a, tc_useful_cols_2021),
                            outcome_id = park_file_2019, outcome_useful_cols = park_useful_cols_2019)
save(dat_park, file = "data_formatted/dat_park.RData")

In [ ]:
# Reproduction T2D
res_list_t2d <- global_uv(api_use_ex = FALSE, api_use_out = FALSE, 
                            exposure_id = t2d_file_2018a, outcome_id = als_file_2018, 
                            exposure_useful_cols = t2d_useful_cols_2018a, 
                            outcome_useful_cols = als_useful_cols_2018,
                            snplist = snpList_t2d, paper_results = "t2d_results.csv")
save(res_list_t2d, file = "data_formatted/res_list_t2d.RData")

In [ ]:
# Reproduction TC
res_list_tc <- global_uv(api_use_ex = FALSE, api_use_out = FALSE, 
                            exposure_id = tc_file_2013, outcome_id = als_file_2021, 
                            exposure_useful_cols = tc_useful_cols_2013, 
                            outcome_useful_cols = als_useful_cols_2021,
                            paper_results = "tc_results.csv")
save(res_list_tc, file = "data_formatted/res_list_tc.RData")

In [ ]:
# Reproduction BMI
dat_bmi <- data_import(api_use_ex = FALSE, api_use_out = FALSE,
                            exposure_id = bmi_file_2015, outcome_id = als_file_2018, 
                            exposure_useful_cols = bmi_useful_cols_sumstat, outcome_useful_cols = als_useful_cols_2018, 
                            snplist = snplist_bmi)
res_bmi <- run_all_mr(dat_bmi)
res_bmi <- plotting_and_sensitivity_analysis(res_bmi, dat_bmi, FALSE)
res_list_bmi <- list(data = dat_bmi, result = res_bmi)
save(res_list_bmi, file = "data_formatted/res_list_bmi.RData")

## Load data

In [5]:
load("data_formatted/dat_als.RData", verbose = TRUE)
load("data_formatted/dat_park.RData", verbose = TRUE)

load("data_formatted/res_list_t2d.RData", verbose = TRUE)
load("data_formatted/res_list_tc.RData", verbose = TRUE)
load("data_formatted/res_list_bmi.RData", verbose = TRUE)

Loading objects:
  dat_als
Loading objects:
  dat_park
Loading objects:
  res_list_t2d
Loading objects:
  res_list_tc
Loading objects:
  res_list_bmi


## Processing ALS

In [83]:
plot_als <- function(mvdat, uvdat_t2d, uvdat_tc, uvdat_bmi, paper_res_files) {
    mv_dat <- mvdat$mv_dat
    uv_dat <- mvdat$uv_dat

    df <- bind_rows(uvdat_t2d$result, uvdat_tc$result, uvdat_bmi$result)

    method_names <- c("Paper's results for IVW", "Replication of IVW analysis from study's GWAS", "MVMR-IVW results on new GWAS")

    df <- subset(df, method == "Inverse variance weighted")
    df$replication <- method_names[2]

    paper_df <- lapply(paper_res_files, function(x) {
                row <- fread(paste0(data_dir, "/paper-results/", x)) %>%
                subset(method == "Inverse variance weighted")
                if (!"or" %in% colnames(row)) {row <- generate_odds_ratios(row)}
                return(row)
                }) %>% 
                bind_rows()
    paper_df$replication <- method_names[1]

    df <- bind_rows(df, paper_df) %>%
    fill(c("id.outcome", "outcome"), .direction = "down")

    res_mv <- TwoSampleMR::mv_multiple(mv_dat, intercept = FALSE, instrument_specific = FALSE, 5e-8)$result %>% 
        generate_odds_ratios()
    res_mv$method <- "MVMR-IVW"
    res_mv$replication <- method_names[3]

    df$exposure <- factor(sub(" \\|.*", "", df$exposure))
    res_mv$exposure <- factor(sub(" \\|.*", "", res_mv$exposure))

    df <- bind_rows(df, res_mv)
    df$replication <- factor(df$replication, levels = method_names)

    ## Plotting
    pos <- position_dodge(width = -.5)

    p <- ggplot(df, aes(x = or, y = exposure, group = replication, color = replication)) + 
        geom_vline(aes(xintercept = 1.0), linewidth = .25, linetype = "dashed") + 
        geom_errorbarh(aes(xmax = or_uci95, xmin = or_lci95), size = .7, height = .2, position=pos) +
        geom_point(position=pos, size = 3) +
        
        scale_x_continuous(breaks = c(0.6,0.8,0.9,1.0,1.1,1.2,1.4), labels = c(0.6,0.8,0.9,1.0,1.1,1.2,1.4), limits = c(0.55,1.45)) +
        
        ylab("") +
        xlab("Odds ratio") +

        scale_colour_manual(values = c("#1b651b", "#00429d", "#93003a")) +
        guides(colour = guide_legend(title = "Replicated", reverse = FALSE)) +
        
        #annotate(geom = "text", y =1.1, x = log10(1.5), label = "", size = 1.5, hjust = 0) + 
        ggtitle("Comparison MR results between papers, replication and MVMR results on ALS") +

        theme_bw() +
        theme(plot.title = element_text(hjust = 0.25))

    return(p)
}

## Processing Park

In [81]:
plot_park <- function(dat, paper_res_file) {
    mv_dat <- dat$mv_dat
    uv_dat <- dat$uv_dat

    method_names <- c("Paper's results for IVW", "IVW analysis on new GWAS", "MR-Egger analysis on new GWAS", "MVMR-IVW analysis on new GWAS")

    res_list_park <- lapply(uv_dat, function(x) {
        x <- filter(x, mr_keep == TRUE)
        res_ivw <- TwoSampleMR::mr_ivw(b_exp = x$beta.exposure, b_out = x$beta.outcome, 
                                    se_exp = x$se.exposure, se_out = x$se.outcome) %>% 
            generate_odds_ratios()
        res_ivw$exposure <- x$exposure[1]
        res_ivw$replication <- method_names[2]
        res_ivw$method <- "Inverse variance weighted"
        return(res_ivw)
    })
    df <- bind_rows(res_list_park)

    res_park_egger <- lapply(uv_dat, function(x) {
        x <- filter(x, mr_keep == TRUE)
        res_egger <- mr_egger_regression(b_exp = x$beta.exposure, b_out = x$beta.outcome, 
                                        se_exp = x$se.exposure, se_out = x$se.outcome) 
        data.frame(method = "MR-Egger",
                    nsnp = res_egger$nsnp,
                    b = res_egger$b,
                    se = res_egger$se,
                    pval = res_egger$pval,
                    exposure = x$exposure[1],
                    replication = method_names[3]) %>% 
            generate_odds_ratios()
    }) %>% bind_rows()

    paper_df <- fread(paste0(data_dir, "/paper-results/", paper_res_file))
    paper_df$replication <- method_names[1]

    df <- bind_rows(df, res_park_egger, paper_df)

    res_mv <- TwoSampleMR::mv_multiple(mv_dat, intercept = FALSE, instrument_specific = FALSE, 5e-8)$result %>% 
            generate_odds_ratios()
    res_mv$method <- "MVMR-IVW"
    res_mv$replication <- method_names[4]

    df <- bind_rows(df, res_mv)
    df$exposure <- factor(sub(" \\|.*", "", df$exposure))

    df$replication <- factor(df$replication, levels = method_names)
    df <- data.frame(df)

    ## Plotting
    pos <- position_dodge(width = -.5)

    p <- ggplot(df, aes(x = or, y = exposure, group = replication, color = replication)) + 
        geom_vline(aes(xintercept = 1.0), linewidth = .25, linetype = "dashed") + 
        geom_errorbarh(aes(xmax = or_uci95, xmin = or_lci95), size = .7, height = .2, position=pos) +
        geom_point(position=pos, size = 3) +
        
        scale_x_continuous(breaks = c(0.6,0.8,0.9,1.0,1.1,1.2,1.4), labels = c(0.6,0.8,0.9,1.0,1.1,1.2,1.4), limits = c(0.55,1.45)) +
        
        ylab("") +
        xlab("Odds ratio") +

        scale_colour_manual(values = c("#1b651b", "#00429d", '#ffa600', "#93003a")) +
        guides(colour = guide_legend(title = "Replicated", reverse = FALSE)) +
        
        #annotate(geom = "text", y =1.1, x = log10(1.5), label = "", size = 1.5, hjust = 0) + 
        ggtitle("Comparison of MR results between paper, replication and MVMR for Parkinson") +

        theme_bw() +
        theme(plot.title = element_text(hjust = 0.3))

    return(p)
}

## MVMR Sensitivity ALS

In [85]:
plot_mv_sensitivity <- function(dat) {
    mv_dat <- dat$mv_dat
    method_names <- c("MVMR-IVW", "MVMR-robust", "MVMR-Egger")

    res_mv <- TwoSampleMR::mv_multiple(mv_dat, intercept = TRUE, instrument_specific = FALSE, 5e-8)$result %>%
        mutate(method = method_names[1], .) %>%
        select(method, everything()) %>%
        generate_odds_ratios()
    
    res_robust <- mvmr_robust(mv_dat) %>%
        mutate(method = method_names[2], .) %>%
        select(method, everything()) %>%
        generate_odds_ratios()
        
    res_egger <- mvmr_egger(mv_dat) %>%
        mutate(method = method_names[3], .) %>%
        select(method, everything()) %>%
        generate_odds_ratios()

    df <- bind_rows(res_mv, res_robust, res_egger)
    df$exposure <- factor(sub(" \\|.*", "", df$exposure))
    df$method <- factor(df$method, levels = method_names)

    res_robust$mv <- TRUE
    res_mv$mv <- FALSE
    res_egger$mv <- TRUE

    export_mvmr_plot(res_robust, res_mv, FALSE,
                    model_names = c('TRUE' = "MVMR-Robust", 'FALSE' = "MVMR-IVW"),
                    colors = c("purple", "blue"))
                    
    export_mvmr_plot(res_egger, res_mv, FALSE,
                    model_names = c('TRUE' = "MVMR-Egger", 'FALSE' = "MVMR-IVW"),
                    colors = c("darkgreen", "blue"))
    
    ## Plotting
    pos <- position_dodge(width = -.5)

    p <- ggplot(df, aes(x = or, y = exposure, group = method, color = method)) + 
        geom_vline(aes(xintercept = 1.0), linewidth = .25, linetype = "dashed") + 
        geom_errorbarh(aes(xmax = or_uci95, xmin = or_lci95), size = .7, height = .2, position=pos) +
        geom_point(position=pos, size = 3) +

        scale_x_continuous(breaks = c(0.6,0.8,0.9,1.0,1.1,1.2,1.4), labels = c(0.6,0.8,0.9,1.0,1.1,1.2,1.4), limits = c(0.55,1.45)) +
        
        ylab("") +
        xlab("Odds ratio") +

        scale_colour_manual(values = c("#00429d", '#ffa600', "#93003a")) +
        guides(colour = guide_legend(title = "Replicated", reverse = FALSE)) +
        
        #annotate(geom = "text", y =1.1, x = log10(1.5), label = "", size = 1.5, hjust = 0) + 
        ggtitle("Comparison of sensitivity analysis for MVMR results on ALS") +

        theme_bw() +
        theme(plot.title = element_text(hjust = 0.1))

    return(p)
}

## Plotting

In [100]:
p1 <- plot_als(dat_als, res_list_t2d, res_list_tc, res_list_bmi, 
                   paper_res_files = c("t2d_results.csv", "tc_results.csv", "bmi_results.csv"))
p2 <- plot_park(dat_park, paper_res_file = "park_results.csv")
#p_final <- grid.arrange(p1, p2, ncol = 2, respect = TRUE)
p_final <- p1 + p2 + patchwork::plot_layout(2, 1)

output_width <- 15  # Width in inches 
output_height <- 6  # Height in inches
ggsave("plot_als_park.png", p_final, width = output_width, height = output_height, dpi = 300)

Warning message:
“`position_dodge()` requires non-overlapping x intervals.”
Warning message:
“`position_dodge()` requires non-overlapping x intervals.”


In [101]:
p1 <- plot_als(dat_als, res_list_t2d, res_list_tc, res_list_bmi, 
                   paper_res_files = c("t2d_results.csv", "tc_results.csv", "bmi_results.csv"))
p2 <- plot_mv_sensitivity(dat_als)
p_final <- p1 + p2 + patchwork::plot_layout(2, 1)

output_width <- 15  # Width in inches 
output_height <- 6  # Height in inches
ggsave("plot_als_sensi.png", p_final, width = output_width, height = output_height, dpi = 300)

Adding missing grouping variables: `id.exposure`
Adding missing grouping variables: `id.exposure`
Warning message:
“`position_dodge()` requires non-overlapping x intervals.”
Warning message:
“`position_dodge()` requires non-overlapping x intervals.”


## Conditionnal Weak Instrument Test

In [130]:
conditionnal_weak_iv_test <- function(mvdat) {
    mv_dat <- mvdat$mv_dat
    mvmr_dat <- MVMR::format_mvmr(BXGs = mv_dat$exposure_beta, BYG = mv_dat$outcome_beta, 
                                    seBXGs = mv_dat$exposure_se, seBYG = mv_dat$outcome_se,
                                    RSID = attributes(mv_dat$exposure_beta)$dimnames[[1]])

    ## Estimating phenotypic correlation
    b <- mv_dat$exposure_beta
    se <- mv_dat$exposure_se
    rsid <- attributes(mv_dat$exposure_beta)$dimnames[[1]]

    allele_dat <- unique(bind_rows(lapply(mvdat$uv_dat, function(df) {
        id <- df$id.exposure[1]
        # Extract SNP data selected in MV analysis
        df <- df[,c("SNP", "effect_allele.exposure", "other_allele.exposure", "beta.exposure")] %>%
            filter(SNP %in% rsid)
        beta_mv <- b[rownames(b) %in% df$SNP, id]
        # Check if alleles need to be flipped
        df$flip <- (df$beta.exposure * beta_mv) < 0
        # Swap alleles if necessary
        df %>%
            mutate(effect_allele.exposure_new = ifelse(flip, other_allele.exposure, effect_allele.exposure),
                other_allele.exposure = ifelse(flip, effect_allele.exposure, other_allele.exposure),
                effect_allele.exposure = effect_allele.exposure_new) %>%
            select(-effect_allele.exposure_new, -beta.exposure, -flip)
        })))
    rownames(allele_dat) <- allele_dat$SNP
    allele_dat$SNP <- NULL
    colnames(allele_dat) <- c("allele_0", "allele_1")

    colnames(b) <- lapply(1:dim(b)[2], function(i) paste0("trait", i, '_b'))
    colnames(se) <- lapply(1:dim(b)[2], function(i) paste0("trait", i, '_se'))

    # Create a new data frame by interleaving columns of b and se
    interleaved_df <- data.frame(matrix(ncol = 2 * ncol(b), nrow = nrow(b)))

    # Assign column names
    colnames(interleaved_df) <- as.vector(rbind(colnames(b), colnames(se)))

    # Fill the interleaved data frame
    for (i in 1:ncol(b)) {
        interleaved_df[, 2 * i - 1] <- b[, i]
        interleaved_df[, 2 * i] <- se[, i]
    }

    S_XY <- bind_cols(allele_dat, interleaved_df)
    S_XY$allele_0 <- as.factor(S_XY$allele_0)
    S_XY$allele_1 <- as.factor(S_XY$allele_1)

    ##estimate phenotypic correlation matrix using metaCCA 
    S_YY <- estimateSyy(S_XY = S_XY)

    Xcovmat <- MVMR::phenocov_mvmr(Pcov = S_YY, seBXGs = mvmr_dat[, c('sebetaX1', 'sebetaX2', 'sebetaX3')]) %>%
        lapply(FUN = as.matrix)
        
    f_stat <- data.frame(t(strength_mvmr(r_input = mvmr_dat, gencov = Xcovmat)))
    q_stat <- pleiotropy_mvmr(r_input = mvmr_dat, gencov = Xcovmat)

    f_stat$Q <- q_stat$Qstat
    f_stat$Q_pval <- q_stat$Qpval

    exp_names <- mv_dat$expname
    rownames(exp_names) <- lapply(1:dim(exp_names)[1], function(i) paste0("exposure", i))
    sensi_stat <- bind_cols(exp_names,f_stat)
    rownames(sensi_stat) <- NULL

    return(list(sensi_stat, S_YY, mvmr_dat))
}

In [131]:
stat_list <- conditionnal_weak_iv_test(dat_als)


Conditional F-statistics for instrument strength

            exposure1 exposure2 exposure3
F-statistic  39.22337  61.84805  11.49867
Q-Statistic for instrument validity:
454.0461 on 319 DF , p-value: 9.569603e-07


In [133]:
# Test Mendelian randomization model adjusting for weak instruments
qhet_res <- qhet_mvmr(r_input = stat_list[[3]], pcor = stat_list[[2]], CI = TRUE, iterations = 100)

Warning message in qhet_mvmr(r_input = stat_list[[3]], pcor = stat_list[[2]], CI = TRUE, :
“qhet_mvmr() is currently undergoing development.”
